In [ ]:
## Last did XGBoost

In [1]:
# Feature normalisation
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')
import pickle
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [2]:
# Read the saved model using pickle
# with open("RandomForestClassifier.pickle", "rb") as f:
#     random_forest_classifier = pickle.load(f)
    
with open("XGBoostClassifier.pickle", "rb") as f:
    xgboost_classifier = pickle.load(f)

In [3]:
test_df = pd.read_csv("Test_Set_Values.csv")

In [4]:
# Store the 'ID' column
ID_data = test_df['id']

In [5]:
test_df['longitude_imputation'] = test_df['longitude']
test_df['latitude_imputation'] = test_df['latitude']

test_df['longitude_imputation'] = test_df['longitude_imputation'].replace(0, np.nan)
test_df['longitude_imputation'] = test_df['longitude_imputation'].fillna(test_df.groupby('region')['longitude_imputation'].transform('mean'))

test_df['latitude_imputation'] = test_df['latitude_imputation'].replace(-2.000000e-08, np.nan)
test_df['latitude_imputation'] = test_df['latitude_imputation'].fillna(test_df.groupby('region')['latitude_imputation'].transform('mean'))

In [6]:
test_df['recorded_year'] = pd.DatetimeIndex(test_df ['date_recorded']).year
test_df['age'] = test_df['recorded_year'] - test_df['construction_year']
test_df['age'] = test_df['age'].apply(lambda x: x if x < 100 else -1)
test_df = test_df.drop('recorded_year',axis=1)

In [7]:
test_df['funder'] = test_df['funder'].replace(['Kkkt_makwale', 'Kkkt Church','Kkkt-dioces Ya Pare','Kkkt Dme','Kkkt Ndrumangeni',
                      'Kkkt Canal','Kkkt Mareu','Kkkt Usa','Kkkt Leguruki'], 'Kkkt')
test_df['funder'] = test_df['funder'].replace(['Hhesawa'], 'Hesawa')
test_df['funder'] = test_df['funder'].replace(['Nrwssp','Rwssp Shinyanga','Drwssp'], 'Rwssp')
test_df['funder'] = test_df['funder'].replace(['Tasafu','Tasaf 1','Tasaf Ii'], 'Tasaf')
test_df['funder'] = test_df['funder'].replace(['Kilindi District Co','Songea District Council','Sengerema District Council','Mbozi District Council',
                                                       'Sangea District Council','Cdtfdistrict Council','Mkinga  Distric Cou'], 'District Council')
test_df['funder'] = test_df['funder'].replace(['Dhv Moro'], 'Dhv')
test_df['funder'] = test_df['funder'].replace(['Germany Republi','Germany'], 'Germany')
test_df['funder'] = test_df['funder'].replace(['Tcrs Kibondo','Tcrst'], 'Tcrs')
test_df['funder'] = test_df['funder'].replace(['Netherland'], 'Netherlands')
test_df['funder'] = test_df['funder'].replace(['Ruangwa Lga'], 'Lga')
test_df['funder'] = test_df['funder'].replace(['Amrefe'], 'Amref')
test_df['funder'] = test_df['funder'].replace(['Oxfam Gb'], 'Oxfam')
test_df['funder'] = test_df['funder'].replace(['Unicet'], 'Unicef')
test_df['funder'] = test_df['funder'].replace(['Rc Church','Rc','Rc Ch','Rc Churc','Rc Mission','Rc Cathoric','Rc Njoro','Rc Mofu','Rc Mi','Rc Missionary',
                                                       'Rc Missi','Rc Msufi'], 'RC')
test_df['funder'] = test_df['funder'].replace(['Missionaries','Missionary','Neemia Mission','German Missionary','Germany Missionary','Cpps Mission',
                                                      'Heri Mission'], 'Mission')
test_df['funder'] = test_df['funder'].replace(['Private Owned','Private Institutions','Private Co','Private Person','Private Individul'], 'Private')
test_df['funder'] = test_df['funder'].replace(['Roman Catholic','Roman Cathoric-same','Roman Cathoric Same','Roman Church','Roman Catholic Rulenge Diocese',
                                                      'Roman Cathoric -kilomeni','Roman Ca','Roman Cathoric','Roman Cathoric Church'], 'Roman')
test_df['funder'] = test_df['funder'].replace(['Rural Water Supply And Sanitat','Rural Water Supply And Sanita','Rural Water Supply','Rural Water Department'], 'Rural Water Supply')
test_df['funder'] = test_df['funder'].replace(['Ces (gmbh)'], 'Ces(gmbh)')
test_df['funder'] = test_df['funder'].replace(['Dwsdp'], 'WSDP')
test_df['funder'] = test_df['funder'].replace(['Finwater'], 'Finw')
test_df['funder'] = test_df['funder'].replace(['Plan International','Plan Internatio'], 'Plan Int')
test_df['funder'] = test_df['funder'].replace(['Oikos E.Afrika'], 'Oikos')
test_df['funder'] = test_df['funder'].replace(['Concern World Wide'], 'Concern')
test_df['funder'] = test_df['funder'].replace(['African Development Bank','African Relie','African Muslim Agency','African Development Foundation','African Barrick Gold',
                                                      'African Realief Committe Of Ku','African 2000 Network','African Reflections Foundation'], 'African')
test_df['funder'] = test_df['funder'].replace(['Snv Ltd','Snv-swash'], 'Snv')
test_df['funder'] = test_df['funder'].replace(['Villagers Mpi'], 'Villagers')
test_df['funder'] = test_df['funder'].replace(['Halmashauri Ya Wilaya Sikonge','Halmashauri Ya Manispa Tabora','Halmashauri Ya Wilaya','Halmashauri Wil','Gra Na Halmashauri'], 'Halmashauri')

In [8]:
def label_funder (row):
    if row['funder'] in ['Government Of Tanzania']:
        return "GOV"
    if row['funder'] in ['Unknown']:
        return "Unknown"
    if row['funder'] in ['Danida']:
        return "DANIDA"
    if row['funder'] in ['Hesawa']:
        return "HESAWA"
    if row['funder'] in ['Rwssp']:
        return "RWSSP"
    if row['funder'] in ['World Bank']:
        return "WORLD BANK"
    if row['funder'] in ['Kkkt']:
        return "ELCT"
    if row['funder'] in ['World Vision']:
        return "WORLD VISION"
    if row['funder'] in ['Unicef']:
        return "UNICEF"
    if row['funder'] in ['District Council']:
        return "DISTRICT COUNCIL"
    return "OTHER"

test_df['funder_grouped'] = test_df.apply(lambda row: label_funder(row), axis=1)

In [9]:
# Author [Jason]

test_df['installer'] = test_df['installer'].replace(['Central government','Central Government','GOVERNMENT',
                                                             'Tanzania Government','central government','CENTRAL GOVERNMENT',
                                                            'Cental Government','Cebtral Government','Tanzanian Government','Centra Government',
                                                            'Tanzania government','TANZANIAN GOVERNMENT','Gover','Gove','Centr','Central govt','Centra govt'], 'Government')
test_df['installer'] = test_df['installer'].replace(['Commu','Communit','Sekei village community','Village Community','Olgilai village community','Village community','COMMU','Ngiresi village community',
                                                             'Oldadai village community','COMMUNITY','Ilwilo community','Maseka community','commu','Marumbo Community'], 'Community')
test_df['installer'] = test_df['installer'].replace(['DANIDA CO'], 'DANIDA')
test_df['installer'] = test_df['installer'].replace(['KKKT _ Konde and DWE','KKKT CHURCH','KkKT','KKKT-Dioces ya Pare','KKKT DME','Kkkt','KKKT Canal','KKKT Ndrumangeni','KKKT Kilinga','KKKT Katiti juu','KKKT MAREU','KKKT Leguruki'], 'KKKT')
test_df['installer'] = test_df['installer'].replace(['HESAWA','hesawa','HesaWa'], 'Hesawa')
test_df['installer'] = test_df['installer'].replace(['TCRS a','TCRS TWESA'], 'TCRS')
test_df['installer'] = test_df['installer'].replace(['TCRS a','TCRS TWESA'], 'TCRS')
test_df['installer'] = test_df['installer'].replace(['Danid','DANIDS'], 'DANID')
test_df['installer'] = test_df['installer'].replace(['District council','DISTRICT COUNCIL','District COUNCIL','Mbozi District Council','MBULU DISTRICT COUNCIL'], 'District Council')
test_df['installer'] = test_df['installer'].replace(['Lga'], 'LGA')
test_df['installer'] = test_df['installer'].replace(['world vision','WORLD VISION','world vision'], 'World Vision')
test_df['installer'] = test_df['installer'].replace(['Wedeco'], 'WEDECO')
test_df['installer'] = test_df['installer'].replace(['Amref'], 'AMREF')
test_df['installer'] = test_df['installer'].replace(['WUA','WUS'], 'WU')
test_df['installer'] = test_df['installer'].replace(['SEMA CO LTD','SEMA Consultant'], 'SEMA')
test_df['installer'] = test_df['installer'].replace(['da'], 'DA')
test_df['installer'] = test_df['installer'].replace(['Unicef'], 'UNICEF')
test_df['installer'] = test_df['installer'].replace(['Idara ya maji','Idara ya Maji'], 'Idara Ya Maji')
test_df['installer'] = test_df['installer'].replace(['sengerema Water Department','sengerema water Department'], 'Sengerema Water Department')
test_df['installer'] = test_df['installer'].replace(['Kiliwater r'], 'Kiliwater')
test_df['installer'] = test_df['installer'].replace(['NORAD/','Norad'], 'NORAD')
test_df['installer'] = test_df['installer'].replace(['Finwater','FinWater','FinWate'], 'Finw')
test_df['installer'] = test_df['installer'].replace(['villagers','VILLAGERS','Engarasero villagers'], 'Villagers')
test_df['installer'] = test_df['installer'].replace(['Distri','District water department','District Water Department','District water depar','Distric Water Department',], 'District')
test_df['installer'] = test_df['installer'].replace(['Magadini Makiwaru wa'], 'Magadini-Makiwaru wa')
test_df['installer'] = test_df['installer'].replace(['MWE &'], 'MWE')
test_df['installer'] = test_df['installer'].replace(['Handeni Trunk Main('], 'Handeni Trunk Main')
test_df['installer'] = test_df['installer'].replace(['Fini water','FINI WATER','FiNI WATER','FINI Water'], 'Fini Water')
test_df['installer'] = test_df['installer'].replace(['Shipo','SHIPO CONSTRUCTORS'], 'SHIPO')
test_df['installer'] = test_df['installer'].replace(['private','Private person','Private owned','PRIVATE INSTITUTIONS','Private Technician','Private individuals','Private company'], 'Private')
test_df['installer'] = test_df['installer'].replace(['KUWAIT','kuwait','Kuwaiti'], 'Kuwait')
test_df['installer'] = test_df['installer'].replace(['Jaica','JAICA CO'], 'JAICA')

In [10]:
# Author [Jason]

# # Bin Values >1000 as individual Category,
# # <1000 placed in others

def label_installer (row):
    if row['installer'] in ['DWE']:
        return "DWE"
    if row['installer'] in ['Unknown']:
        return "Unknown"
    if row['installer'] in ['Government']:
        return "GOV"
    if row['installer'] in ['Community']:
        return "COMMUNITY"
    if row['installer'] in ['Hesawa']:
        return "HESAWA"
    if row['installer'] in ['RWE']:
        return "RWE"
    if row['installer'] in ['DANIDA']:
        return "DANIDA"
    if row['installer'] in ['KKKT']:
        return "ELCT"
    return "OTHER"

test_df['installer_grouped'] = test_df.apply(lambda row: label_installer(row), axis=1)

In [11]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [12]:
# Author [Jason]

# Function for Region Column Binning. 
# Binning done based on the decade.

def label_construction_year (row):
    if row['construction_year'] in range(1960,1970):
        return "60s"
    if row['construction_year'] in range(1970,1980):
        return "70s"
    if row['construction_year'] in range(1980,1990):
        return "80s"
    if row['construction_year'] in range(1990,2000):
        return "90s"
    if row['construction_year'] in range(2000,2010):
        return "2000s"
    if row['construction_year'] in range(2010, 2020):
        return "2010s"
    return "Unknown"

test_df['construction_decade'] = test_df.apply(lambda row: label_construction_year(row), axis=1)

In [13]:
test_df['public_meeting'] = test_df['public_meeting'].fillna("Unknown")

In [14]:
test_df['permit'] = test_df['permit'].astype(str)
test_df['public_meeting'] = test_df['public_meeting'].astype(str)

In [15]:
# test_df = MultiColumnLabelEncoder(columns = ['quantity','extraction_type_class','waterpoint_type_group','payment_type','source_type','funder_grouped']).fit_transform(test_df)

test_df = MultiColumnLabelEncoder(columns = ['quantity','extraction_type_class','waterpoint_type_group','payment_type','source_type','funder_grouped',
                                            'scheme_management','construction_decade','basin','installer_grouped','management','water_quality','public_meeting']).fit_transform(test_df)

In [16]:
# feature_selected_df = test_df[['longitude_imputation','latitude_imputation','quantity','extraction_type_class','gps_height','age',
#                                'waterpoint_type_group','population','payment_type','source_type','amount_tsh','funder_grouped']]

feature_selected_df = test_df[['longitude_imputation','latitude_imputation','quantity','extraction_type_class','gps_height','age',
                              'waterpoint_type_group','population','payment_type','source_type','funder_grouped',
                              'basin','installer_grouped','management','water_quality','public_meeting']]

In [17]:
scaler = StandardScaler()
Xs_predict = scaler.fit_transform(feature_selected_df)

In [18]:
# testing_predictions = random_forest_classifier.predict(Xs_predict)

testing_predictions = xgboost_classifier.predict(Xs_predict)

In [19]:
target_df = pd.DataFrame({'ID': ID_data, 'status_group': testing_predictions})
target_df = target_df.replace({'status_group' : { 0 : "functional", 1 : "functional needs repair", 2 : "non functional" } } )
target_df.to_excel("Submission_V5.xls")